## Extract Dataset

In [ ]:
import pandas as pd
import urllib.request, json
from sqlalchemy import create_engine
from tabulate import tabulate
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [ ]:
city_url = os.getenv("CITY_URL")
country_url = city_url = os.getenv("COUNTRY_URL")
requirements_table_url = os.getenv("REQUIREMENT_URL")

city_raw = pd.read_csv(city_url)
country_raw = pd.read_csv(country_url)

In [95]:
with urllib.request.urlopen(requirements_table_url) as url:
    requirements_table = json.load(url)

requirements_table

{'actor': [{'column_name': 'actor_id', 'data_type': 'int64'},
  {'column_name': 'last_update', 'data_type': 'datetime64[ns]'},
  {'column_name': 'first_name', 'data_type': 'object'},
  {'column_name': 'last_name', 'data_type': 'object'}],
 'store': [{'column_name': 'store_id', 'data_type': 'int64'},
  {'column_name': 'manager_staff_id', 'data_type': 'int64'},
  {'column_name': 'address_id', 'data_type': 'int64'},
  {'column_name': 'last_update', 'data_type': 'datetime64[ns]'}],
 'address': [{'column_name': 'last_update', 'data_type': 'datetime64[ns]'},
  {'column_name': 'city_id', 'data_type': 'int64'},
  {'column_name': 'address_id', 'data_type': 'int64'},
  {'column_name': 'district', 'data_type': 'object'},
  {'column_name': 'phone', 'data_type': 'object'},
  {'column_name': 'postal_code', 'data_type': 'object'},
  {'column_name': 'address', 'data_type': 'object'},
  {'column_name': 'address2', 'data_type': 'object'}],
 'category': [{'column_name': 'category_id', 'data_type': 'int64

In [ ]:
dbname = os.getenv("DB_NAME_DATA_SOURCE")
user = os.getenv("USER_DATA_SOURCE")
password = os.getenv("PASSWORD_DATA_SOURCE")
host = os.getenv("HOST_DATA_SOURCE")
port = os.getenv("PORT_DATA_SOURCE")

engine_str = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(engine_str)

In [97]:
def get_table_data(table_name, engine):
    try:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)

        return df
    except Exception as e:
        print(f"Error: {e}")

        return pd.DataFrame()

In [98]:
actor_df = get_table_data('actor', engine)
store_df = get_table_data('store', engine)
address_df = get_table_data('address', engine)
category_df = get_table_data('category', engine)
customer_df = get_table_data('customer', engine)
film_actor_df = get_table_data('film_actor', engine)
film_category_df = get_table_data('film_category', engine)
inventory_df = get_table_data('inventory',engine)
language_df = get_table_data('language',engine)
rental_df = get_table_data('rental',engine)
staff_df = get_table_data('staff',engine)
payment_df = get_table_data('payment',engine)
film_df = get_table_data('film',engine)

In [99]:
table_dict = {
    'actor': actor_df,
    'store': store_df,
    'address': address_df,
    'category': category_df,
    'customer': customer_df,
    'film_actor': film_actor_df,
    'film_category': film_category_df,
    'inventory': inventory_df,
    'language': language_df,
    'rental': rental_df,
    'staff': staff_df,
    'payment': payment_df,
    'film': film_df,
    'city': city_raw,
    'country': country_raw
}

## Read Data

In [100]:
city_raw.head()

,city_id,city,country
0,1,A Corua (La Corua),Spain
1,2,Abha,Saudi Arabia
2,3,Abu Dhabi,United Arab Emirates
3,4,Acua,Mexico
4,5,Adana,Turkey


In [101]:
country_raw.head()

,country,last_update
0,Afghanistan,2006-02-15 09:44:00
1,Algeria,2006-02-15 09:44:00
2,American Samoa,2006-02-15 09:44:00
3,Angola,2006-02-15 09:44:00
4,Anguilla,2006-02-15 09:44:00


In [102]:
actor_df.head()

,actor_id,first_name,last_name,last_update
0,1,Penelope,Guiness,2013-05-26 14:47:57.620
1,2,Nick,Wahlberg,2013-05-26 14:47:57.620
2,3,Ed,Chase,2013-05-26 14:47:57.620
3,4,Jennifer,Davis,2013-05-26 14:47:57.620
4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620


In [103]:
print(len(requirements_table.keys()))
print(len(table_dict))

15
15


## Data Validation

In [104]:

def check_table_requirements(actual_table, requirement_table):
    # column name checker
    actual_table_name = [table for table in actual_table]
    requirement_table_name = list(requirement_table.keys())
    table_checking = []

    for table_name in actual_table_name:
        if table_name in requirement_table_name:
            table_checking.append([table_name, 'v'])
        else:
            table_checking.append([table_name, 'x'])

    table_headers = ['table_name', 'is_exist']
    table = tabulate(table_checking, headers=table_headers, tablefmt='grid')
    print('=> STEP 1: Check Table')
    print(table)

In [105]:
check_table_requirements(actual_table=table_dict, requirement_table=requirements_table)

=> STEP 1: Check Table
+---------------+------------+
| table_name    | is_exist   |
+===============+============+
| actor         | v          |
+---------------+------------+
| store         | v          |
+---------------+------------+
| address       | v          |
+---------------+------------+
| category      | v          |
+---------------+------------+
| customer      | v          |
+---------------+------------+
| film_actor    | v          |
+---------------+------------+
| film_category | v          |
+---------------+------------+
| inventory     | v          |
+---------------+------------+
| language      | v          |
+---------------+------------+
| rental        | v          |
+---------------+------------+
| staff         | v          |
+---------------+------------+
| payment       | v          |
+---------------+------------+
| film          | v          |
+---------------+------------+
| city          | v          |
+---------------+------------+
| country       

In [106]:
def check_shape(actual_table):

    table_shape = []

    for table, data in actual_table.items():
        table_shape.append([table, data.shape[0], data.shape[1]])

    table_headers = ['table_name', 'Number of rows', 'Number of Columns']
    table = tabulate(table_shape, headers=table_headers, tablefmt='grid')
    print('=> STEP 2: Check Data Shape')
    print(table)

In [107]:
check_shape(actual_table=table_dict)

=> STEP 2: Check Data Shape
+---------------+------------------+---------------------+
| table_name    |   Number of rows |   Number of Columns |
+===============+==================+=====================+
| actor         |              200 |                   4 |
+---------------+------------------+---------------------+
| store         |                2 |                   4 |
+---------------+------------------+---------------------+
| address       |              603 |                   8 |
+---------------+------------------+---------------------+
| category      |               16 |                   3 |
+---------------+------------------+---------------------+
| customer      |              599 |                  10 |
+---------------+------------------+---------------------+
| film_actor    |             5462 |                   3 |
+---------------+------------------+---------------------+
| film_category |             1000 |                   3 |
+---------------+-----------

In [108]:
table_dict['actor'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   actor_id     200 non-null    int64         
 1   first_name   200 non-null    object        
 2   last_name    200 non-null    object        
 3   last_update  200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 6.4+ KB


In [109]:
a = ["1", 2, 3, 4, 5]
b = ['j', 2, 3, 4, 5]
print(set(a) & set(b))

{2, 3, 4, 5}


In [110]:
columns_actual = table_dict['country'].columns
columns_req = [col['column_name'] for col in requirements_table['country']]

print(columns_actual)
print(columns_req)

Index(['country', 'last_update'], dtype='object')
['country_id', 'last_update', 'country']


In [111]:
def check_columns(actual_table, req_table):
    print('=> STEP 3: Check Columns')
    for key in req_table:
        column_names = []
        columns_req = [col['column_name'] for col in req_table[key]]
        columns_actual = actual_table[key].columns

        for column in columns_actual:
            if column in columns_req:
                column_names.append([column, 'v', 'v'])
            else:
                column_names.append([column, 'v', 'x'])

        for column in columns_req:
            if column not in columns_actual:
                column_names.append([column, 'x', 'v'])

        table_headers = ['column_name', 'in_actual_table', 'in_requirements_table']
        table = tabulate(column_names, headers=table_headers, tablefmt='grid') 
        print(f'Table: {key}')
        print(f'{table}\n')


In [112]:
check_columns(actual_table = table_dict, req_table = requirements_table)

=> STEP 3: Check Columns
Table: actor
+---------------+-------------------+-------------------------+
| column_name   | in_actual_table   | in_requirements_table   |
+===============+===================+=========================+
| actor_id      | v                 | v                       |
+---------------+-------------------+-------------------------+
| first_name    | v                 | v                       |
+---------------+-------------------+-------------------------+
| last_name     | v                 | v                       |
+---------------+-------------------+-------------------------+
| last_update   | v                 | v                       |
+---------------+-------------------+-------------------------+

Table: store
+------------------+-------------------+-------------------------+
| column_name      | in_actual_table   | in_requirements_table   |
+==================+===================+=========================+
| store_id         | v                 | v 

In [113]:
columns_type_actual ={}
columns_type_actual['actor'] = [table_dict['actor'][col].dtype for col in table_dict['actor'].columns]
columns_type_actual['actor']

[dtype('int64'), dtype('O'), dtype('O'), dtype('<M8[ns]')]

In [114]:
columns_type_actual ={}
columns_type_actual['actor'] = [str(table_dict['actor'][col].dtype) for col in table_dict['actor'].columns]
columns_type_actual['actor']

['int64', 'object', 'object', 'datetime64[ns]']

In [115]:
columns_type_req = [col['data_type'] for col in requirements_table['actor']]
columns_type_req

['int64', 'datetime64[ns]', 'object', 'object']

In [116]:
requirements_table['actor']

[{'column_name': 'actor_id', 'data_type': 'int64'},
 {'column_name': 'last_update', 'data_type': 'datetime64[ns]'},
 {'column_name': 'first_name', 'data_type': 'object'},
 {'column_name': 'last_name', 'data_type': 'object'}]

In [117]:
def check_data_types(actual_table, requirements_table):
    print("=> STEP 4: Check Data Types")
    summary_data = []

    for table_name, df in actual_table.items():
        if table_name in requirements_table:
            for column_info in requirements_table[table_name]:
                column_name = column_info["column_name"]
                requirements_type = column_info["data_type"]
                
                if column_name in df.columns:
                    actual_type = str(df[column_name].dtype)
                    match = "✔" if actual_type == requirements_type else "X"
                    summary_data.append([table_name, column_name, actual_type, requirements_type, match])
                else:
                    summary_data.append([table_name, column_name, "N/A", requirements_type, "X (Column not found)"])

    headers = ["Table Name", "Column Name", "Actual Type", "Requirements Type", "Match"]

    mismatch_data = [row for row in summary_data if "X" in row[4]]
    
    if mismatch_data:
        print("\nSummary of Mismatches Data Types:")
        print(tabulate(mismatch_data, headers = headers, tablefmt = "grid"))
    
    else:
        print("All Data Types Match")

In [118]:
check_data_types(actual_table = table_dict, requirements_table = requirements_table)

=> STEP 4: Check Data Types

Summary of Mismatches Data Types:
+--------------+---------------+---------------+---------------------+----------------------+
| Table Name   | Column Name   | Actual Type   | Requirements Type   | Match                |
+==============+===============+===============+=====================+======================+
| customer     | create_date   | object        | datetime64[ns]      | X                    |
+--------------+---------------+---------------+---------------------+----------------------+
| city         | country_id    | N/A           | int64               | X (Column not found) |
+--------------+---------------+---------------+---------------------+----------------------+
| city         | last_update   | N/A           | datetime64[ns]      | X (Column not found) |
+--------------+---------------+---------------+---------------------+----------------------+
| country      | country_id    | N/A           | int64               | X (Column not found)

In [ ]:
def check_missing_values(actual_table):
    missing = []
    print('=> STEP 5: Check Missing Values\n')
    print('Missing Value Summary')

    for table_name, data in actual_table.items():
        if data.isna().sum().sum() != 0:
            temp = data.isnull().sum()[data.isnull().any()].reset_index()
            temp.columns = ['column_name', 'missing_count']
            temp['missing_pct'] = ((temp['missing_count'] / len(data)) * 100).round(2)

            for _, row in temp.iterrows():
                missing.append([table_name, row['column_name'], row['missing_count'], row['missing_pct']])

    if missing:
        table_headers = ['Table Name', 'Column Name', 'Missing Value Count', 'Missing Value Percentage']
        table = tabulate(missing, headers=table_headers, tablefmt='grid')
        print(table)
    else:
        print("No missing values found.\n")

In [120]:
check_missing_values(actual_table=table_dict)

=> STEP 5: Check Missing Values

Missing Value Summary
+--------------+---------------+-----------------------+----------------------------+
| Table Name   | Column Name   |   Missing Value Count |   Missing Value Percentage |
+==============+===============+=======================+============================+
| address      | address2      |                     4 |                       0.66 |
+--------------+---------------+-----------------------+----------------------------+
| rental       | return_date   |                   183 |                       1.14 |
+--------------+---------------+-----------------------+----------------------------+
| staff        | picture       |                     1 |                      50    |
+--------------+---------------+-----------------------+----------------------------+
| city         | city          |                    10 |                       1.48 |
+--------------+---------------+-----------------------+----------------------------+

In [121]:
def check_duplicate(actual_table):
    print("=> STEP 6: Check Duplicates Data")
    duplicate_summary = []

    for table_name, data in actual_table.items():
        try:
            duplicate_rows = data[data.duplicated(keep = False)]
            
            if not duplicate_rows.empty:
                duplicate_summary.append([table_name, len(duplicate_rows)])
        except:
            pass

    if duplicate_summary:
        print("Duplicate Data Summary:")
        print(tabulate(duplicate_summary, headers=["Table Name", "Duplicate Rows Count"], tablefmt="grid"))
    else:
        print("No Duplicate Data Found")

check_duplicate(actual_table=table_dict)

=> STEP 6: Check Duplicates Data
Duplicate Data Summary:
+--------------+------------------------+
| Table Name   |   Duplicate Rows Count |
+==============+========================+
| city         |                    154 |
+--------------+------------------------+


## Data Transform

### Data Cleansing

- missmatch column table:
    1. city
    2. country

- missmatch data type:
    1. customer => create_date
    2. city => country_id
    3. city => last_update
    4. country => country_id
    5. country => last_update

- missing values:
    1. address => address2 : 4
    2. rental => return_date : 183
    3. staff => picture : 1
    4. city => city : 10
    5. city => country : 7

- duplicate data:
    1. city => 154

In [122]:
temp_dict = table_dict.copy()

In [123]:
table_dict['actor']

,actor_id,first_name,last_name,last_update
0,1,Penelope,Guiness,2013-05-26 14:47:57.620
1,2,Nick,Wahlberg,2013-05-26 14:47:57.620
2,3,Ed,Chase,2013-05-26 14:47:57.620
3,4,Jennifer,Davis,2013-05-26 14:47:57.620
4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620
...,...,...,...,...
195,196,Bela,Walken,2013-05-26 14:47:57.620
196,197,Reese,West,2013-05-26 14:47:57.620
197,198,Mary,Keitel,2013-05-26 14:47:57.620
198,199,Julia,Fawcett,2013-05-26 14:47:57.620


#### Handling missmatch column table

In [124]:
table_dict['country']

,country,last_update
0,Afghanistan,2006-02-15 09:44:00
1,Algeria,2006-02-15 09:44:00
2,American Samoa,2006-02-15 09:44:00
3,Angola,2006-02-15 09:44:00
4,Anguilla,2006-02-15 09:44:00
...,...,...
104,Vietnam,2006-02-15 09:44:00
105,"Virgin Islands, U.S.",2006-02-15 09:44:00
106,Yemen,2006-02-15 09:44:00
107,Yugoslavia,2006-02-15 09:44:00


In [125]:
table_dict['country'].nunique()

country        109
last_update      1
dtype: int64

==> jumlah baris table country = jumlah country

In [126]:
len(table_dict['country'])

109

In [127]:
list_id = [i for i in range(len(table_dict['country']))]

In [128]:
list_id

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108]

In [129]:
table_dict['country']['country_id'] = list_id

In [130]:
table_dict['country']

,country,last_update,country_id
0,Afghanistan,2006-02-15 09:44:00,0
1,Algeria,2006-02-15 09:44:00,1
2,American Samoa,2006-02-15 09:44:00,2
3,Angola,2006-02-15 09:44:00,3
4,Anguilla,2006-02-15 09:44:00,4
...,...,...,...
104,Vietnam,2006-02-15 09:44:00,104
105,"Virgin Islands, U.S.",2006-02-15 09:44:00,105
106,Yemen,2006-02-15 09:44:00,106
107,Yugoslavia,2006-02-15 09:44:00,107


In [131]:
table_dict['city']['country'].unique()

array(['Spain', 'Saudi Arabia', 'United Arab Emirates', 'Mexico',
       'Turkey', 'Ethiopia', 'Yemen', 'India', 'Japan', 'United States',
       'Bahrain', 'Sudan', 'Italy', 'Argentina', 'Brazil', 'Netherlands',
       'Chile', 'Iran', 'Puerto Rico', 'Israel', 'Turkmenistan',
       'Paraguay', 'Greece', 'Russian Federation', 'China', 'Azerbaijan',
       'Cameroon', 'Brunei', 'Gambia', 'Venezuela', 'Switzerland',
       'Algeria', 'Cambodia', 'Philippines', 'Mozambique', 'Angola',
       'Morocco', 'Nigeria', 'Egypt', 'Nepal', 'South Africa', 'Romania',
       'United Kingdom', 'Slovakia', 'France', 'Colombia', 'Poland',
       'Peru', 'Vietnam', 'French Guiana', 'Taiwan',
       'Virgin Islands, U.S.', 'South Korea', 'Moldova', 'Indonesia',
       'Holy See (Vatican City State)', 'Pakistan', 'Latvia',
       'Bangladesh', 'Germany', 'Bolivia', 'French Polynesia', 'Tuvalu',
       'Canada', 'Austria', 'New Zealand', 'Malaysia', 'Sri Lanka',
       'Kuwait', 'Afghanistan', 'Ukraine',


In [132]:
table_dict['city']

,city_id,city,country
0,1,A Corua (La Corua),Spain
1,2,Abha,Saudi Arabia
2,3,Abu Dhabi,United Arab Emirates
3,4,Acua,Mexico
4,5,Adana,Turkey
...,...,...,...
672,596,NaN,NaN
673,597,NaN,NaN
674,598,NaN,Kazakstan
675,599,NaN,NaN


In [133]:
city_df = table_dict['city'].merge(table_dict['country'], on = 'country', how = 'left')

city_df = city_df[['city_id', 'country_id', 'city', 'last_update', 'country']]

city_df.head()

,city_id,country_id,city,last_update,country
0,1,86.0,A Corua (La Corua),2006-02-15 09:44:00,Spain
1,2,81.0,Abha,2006-02-15 09:44:00,Saudi Arabia
2,3,100.0,Abu Dhabi,2006-02-15 09:44:00,United Arab Emirates
3,4,59.0,Acua,2006-02-15 09:44:00,Mexico
4,5,96.0,Adana,2006-02-15 09:44:00,Turkey


In [134]:
table_dict['country'][table_dict['country']['country'] == 'United Arab Emirates']

,country,last_update,country_id
100,United Arab Emirates,2006-02-15 09:44:00,100


In [135]:
table_dict['city'] = city_df.drop(columns='country')

In [136]:
table_dict['city'].head()

,city_id,country_id,city,last_update
0,1,86.0,A Corua (La Corua),2006-02-15 09:44:00
1,2,81.0,Abha,2006-02-15 09:44:00
2,3,100.0,Abu Dhabi,2006-02-15 09:44:00
3,4,59.0,Acua,2006-02-15 09:44:00
4,5,96.0,Adana,2006-02-15 09:44:00


In [137]:
check_columns(actual_table=table_dict, req_table=requirements_table)

=> STEP 3: Check Columns
Table: actor
+---------------+-------------------+-------------------------+
| column_name   | in_actual_table   | in_requirements_table   |
+===============+===================+=========================+
| actor_id      | v                 | v                       |
+---------------+-------------------+-------------------------+
| first_name    | v                 | v                       |
+---------------+-------------------+-------------------------+
| last_name     | v                 | v                       |
+---------------+-------------------+-------------------------+
| last_update   | v                 | v                       |
+---------------+-------------------+-------------------------+

Table: store
+------------------+-------------------+-------------------------+
| column_name      | in_actual_table   | in_requirements_table   |
+==================+===================+=========================+
| store_id         | v                 | v 

#### Handling missmatch data type

In [138]:
temp_dict_2 = table_dict.copy()

In [144]:
check_data_types(actual_table=table_dict, requirements_table=requirements_table)

=> STEP 4: Check Data Types

Summary of Mismatches Data Types:
+--------------+---------------+---------------+---------------------+---------+
| Table Name   | Column Name   | Actual Type   | Requirements Type   | Match   |
+==============+===============+===============+=====================+=========+
| city         | country_id    | float64       | int64               | X       |
+--------------+---------------+---------------+---------------------+---------+
| city         | last_update   | object        | datetime64[ns]      | X       |
+--------------+---------------+---------------+---------------------+---------+


In [145]:
table_dict['customer']['create_date'].dtype

dtype('<M8[ns]')

In [140]:
table_dict['customer']['create_date'] = pd.to_datetime(table_dict['customer']['create_date'])
table_dict['country']['last_update'] = pd.to_datetime(table_dict['customer']['create_date'])

In [141]:
print(str(table_dict['customer']['create_date'].dtype))
print(str(table_dict['country']['last_update'].dtype))

datetime64[ns]
datetime64[ns]


In [142]:
check_data_types(actual_table=table_dict, requirements_table=requirements_table)

=> STEP 4: Check Data Types

Summary of Mismatches Data Types:
+--------------+---------------+---------------+---------------------+---------+
| Table Name   | Column Name   | Actual Type   | Requirements Type   | Match   |
+==============+===============+===============+=====================+=========+
| city         | country_id    | float64       | int64               | X       |
+--------------+---------------+---------------+---------------------+---------+
| city         | last_update   | object        | datetime64[ns]      | X       |
+--------------+---------------+---------------+---------------------+---------+


In [148]:
table_dict['city']['country_id'].dtype

dtype('float64')

In [149]:
table_dict['city']['country_id'] = table_dict['city']['country_id'].astype('Int64')
table_dict['city']['last_update'] = pd.to_datetime(table_dict['city']['last_update'])

In [150]:
check_data_types(actual_table=table_dict, requirements_table=requirements_table)

=> STEP 4: Check Data Types

Summary of Mismatches Data Types:
+--------------+---------------+---------------+---------------------+---------+
| Table Name   | Column Name   | Actual Type   | Requirements Type   | Match   |
+==============+===============+===============+=====================+=========+
| city         | country_id    | Int64         | int64               | X       |
+--------------+---------------+---------------+---------------------+---------+


I choose Int64 instead of int64 becouse there are null in country_id actual data

#### Handling Missing Values

In [151]:
temp_dict_3 = table_dict.copy()

In [163]:
check_missing_values(actual_table=table_dict)

=> STEP 5: Check Missing Values

Missing Value Summary
+--------------+---------------+-----------------------+----------------------------+
| Table Name   | Column Name   |   Missing Value Count |   Missing Value Percentage |
+==============+===============+=======================+============================+
| address      | address2      |                     4 |                       0.66 |
+--------------+---------------+-----------------------+----------------------------+
| rental       | return_date   |                   183 |                       1.14 |
+--------------+---------------+-----------------------+----------------------------+
| staff        | picture       |                     1 |                      50    |
+--------------+---------------+-----------------------+----------------------------+
| city         | country_id    |                     7 |                       1.03 |
+--------------+---------------+-----------------------+----------------------------+

dari segi domain knowledge, belum bisa ditentukan missing value diganti dengan apa. karena missing value tidak ada yang melebihi 1.5% (kecuali picture yang memang hanya 2 row data saja), tabel yang memiliki missing values, akan dihapus row nya.

In [164]:
def remove_missing_values(actual_table):
    final_actual_data = {}
    for table, data in actual_table.items():
        cleaned_data = data.dropna()
        final_actual_data[table] = cleaned_data.copy()
    
    return final_actual_data

In [165]:
table_dict = remove_missing_values(actual_table=table_dict)

In [166]:
check_missing_values(table_dict)

=> STEP 5: Check Missing Values

Missing Value Summary
No missing values found.



In [169]:
check_data_types(actual_table=table_dict, requirements_table=requirements_table)

=> STEP 4: Check Data Types

Summary of Mismatches Data Types:
+--------------+---------------+---------------+---------------------+---------+
| Table Name   | Column Name   | Actual Type   | Requirements Type   | Match   |
+==============+===============+===============+=====================+=========+
| city         | country_id    | Int64         | int64               | X       |
+--------------+---------------+---------------+---------------------+---------+


In [ ]:
table_dict['city']['country_id'] = table_dict['city']['country_id'].astype('int64') # udah gaada NaN

In [171]:
check_data_types(actual_table=table_dict, requirements_table=requirements_table)

=> STEP 4: Check Data Types
All Data Types Match


#### Handling Data Duplicate

In [172]:
temp_dict_4 = table_dict.copy()

In [173]:
check_duplicate(actual_table=table_dict)

=> STEP 6: Check Duplicates Data
Duplicate Data Summary:
+--------------+------------------------+
| Table Name   |   Duplicate Rows Count |
+==============+========================+
| city         |                    146 |
+--------------+------------------------+


Data duplicate akan dihapus dengan mempertahankan baris pertama

In [174]:
table_dict['city'] = table_dict['city'].drop_duplicates()

In [175]:
check_duplicate(actual_table=table_dict)

=> STEP 6: Check Duplicates Data
No Duplicate Data Found


### Data Manipulation

In [176]:
cleaned_data_temp = table_dict.copy()

Dari tabel-tabel yang ada, dipilih satu use case, yaitu analisis film. Untuk menghandle use case tersebut, dibuat satu tabel baru yaitu `film_list` yang merupakan kombinasi dari beberapa tabel.

Kombinasi tabel tersebut adalah:
1. actor
2. film_actor
3. film
4. film_category
5. category

In [177]:
actor_data = table_dict['actor'].copy()
film_actor_data = table_dict['film_actor'].copy()
film_data = table_dict['film'].copy()
film_category_data = table_dict['film_category'].copy()
category_data = table_dict['category'].copy()

In [179]:
film_category_data.head()

,film_id,category_id,last_update
0,1,6,2006-02-15 10:07:09
1,2,11,2006-02-15 10:07:09
2,3,6,2006-02-15 10:07:09
3,4,11,2006-02-15 10:07:09
4,5,8,2006-02-15 10:07:09


In [180]:
category_data

,category_id,name,last_update
0,1,Action,2006-02-15 09:46:27
1,2,Animation,2006-02-15 09:46:27
2,3,Children,2006-02-15 09:46:27
3,4,Classics,2006-02-15 09:46:27
4,5,Comedy,2006-02-15 09:46:27
5,6,Documentary,2006-02-15 09:46:27
6,7,Drama,2006-02-15 09:46:27
7,8,Family,2006-02-15 09:46:27
8,9,Foreign,2006-02-15 09:46:27
9,10,Games,2006-02-15 09:46:27


In [187]:
film_list = category_data.merge(film_category_data, how='left', on='category_id', suffixes=('_x1', '_y1'))
film_list

,category_id,name,last_update_x1,film_id,last_update_y1
0,1,Action,2006-02-15 09:46:27,19,2006-02-15 10:07:09
1,1,Action,2006-02-15 09:46:27,21,2006-02-15 10:07:09
2,1,Action,2006-02-15 09:46:27,29,2006-02-15 10:07:09
3,1,Action,2006-02-15 09:46:27,38,2006-02-15 10:07:09
4,1,Action,2006-02-15 09:46:27,56,2006-02-15 10:07:09
...,...,...,...,...,...
995,16,Travel,2006-02-15 09:46:27,931,2006-02-15 10:07:09
996,16,Travel,2006-02-15 09:46:27,977,2006-02-15 10:07:09
997,16,Travel,2006-02-15 09:46:27,981,2006-02-15 10:07:09
998,16,Travel,2006-02-15 09:46:27,988,2006-02-15 10:07:09


In [188]:
film_data

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,133,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,2006,1,7,4.99,117,14.99,NC-17,2013-05-26 14:50:58.951,[Trailers],'chamber':1 'fate':4 'husband':11 'italian':2 ...
1,384,Grosse Wonderful,A Epic Drama of a Cat And a Explorer who must ...,2006,1,5,4.99,49,19.99,R,2013-05-26 14:50:58.951,[Behind the Scenes],'australia':18 'cat':8 'drama':5 'epic':4 'exp...
2,8,Airport Pollock,A Epic Tale of a Moose And a Girl who must Con...,2006,1,6,4.99,54,15.99,R,2013-05-26 14:50:58.951,[Trailers],'airport':1 'ancient':18 'confront':14 'epic':...
3,98,Bright Encounters,A Fateful Yarn of a Lumberjack And a Feminist ...,2006,1,4,4.99,73,12.99,PG-13,2013-05-26 14:50:58.951,[Trailers],'boat':20 'bright':1 'conquer':14 'encount':2 ...
4,1,Academy Dinosaur,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'academi':1 'battl':15 'canadian':20 'dinosaur...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Young Language,A Unbelieveable Yarn of a Boat And a Database ...,2006,1,6,0.99,183,9.99,G,2013-05-26 14:50:58.951,"[Trailers, Behind the Scenes]",'administr':12 'boat':8 'boy':17 'databas':11 ...
996,997,Youth Kick,A Touching Drama of a Teacher And a Cat who mu...,2006,1,4,0.99,179,14.99,NC-17,2013-05-26 14:50:58.951,"[Trailers, Behind the Scenes]",'boat':22 'cat':11 'challeng':14 'drama':5 'ki...
997,998,Zhivago Core,A Fateful Yarn of a Composer And a Man who mus...,2006,1,6,0.99,105,10.99,NC-17,2013-05-26 14:50:58.951,[Deleted Scenes],'boy':16 'canadian':19 'compos':8 'core':2 'fa...
998,999,Zoolander Fiction,A Fateful Reflection of a Waitress And a Boat ...,2006,1,5,2.99,101,28.99,R,2013-05-26 14:50:58.951,"[Trailers, Deleted Scenes]",'ancient':19 'boat':11 'china':20 'discov':14 ...


In [189]:
film_list = film_list.merge(film_data, how='left', on='film_id', suffixes=('_x2', '_y2'))
film_list

,category_id,name,last_update_x1,film_id,last_update_y1,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,1,Action,2006-02-15 09:46:27,19,2006-02-15 10:07:09,Amadeus Holy,A Emotional Display of a Pioneer And a Technic...,2006,1,6,0.99,113,20.99,PG,2013-05-26 14:50:58.951,"[Commentaries, Deleted Scenes, Behind the Scenes]",'amadeus':1 'baloon':20 'battl':15 'display':5...
1,1,Action,2006-02-15 09:46:27,21,2006-02-15 10:07:09,American Circus,A Insightful Drama of a Girl And a Astronaut w...,2006,1,3,4.99,129,17.99,R,2013-05-26 14:50:58.951,"[Commentaries, Behind the Scenes]",'administr':17 'american':1 'astronaut':11 'ci...
2,1,Action,2006-02-15 09:46:27,29,2006-02-15 10:07:09,Antitrust Tomatoes,A Fateful Yarn of a Womanizer And a Feminist w...,2006,1,5,2.99,168,11.99,NC-17,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Deleted Scenes]",'administr':17 'ancient':19 'antitrust':1 'dat...
3,1,Action,2006-02-15 09:46:27,38,2006-02-15 10:07:09,Ark Ridgemont,A Beautiful Yarn of a Pioneer And a Monkey who...,2006,1,6,0.99,68,25.99,NC-17,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Deleted Scenes, Behin...",'ark':1 'beauti':4 'desert':20 'explor':16 'mo...
4,1,Action,2006-02-15 09:46:27,56,2006-02-15 10:07:09,Barefoot Manchurian,A Intrepid Story of a Cat And a Student who mu...,2006,1,6,2.99,129,15.99,G,2013-05-26 14:50:58.951,"[Trailers, Commentaries]",'abandon':19 'amus':20 'barefoot':1 'cat':8 'g...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,16,Travel,2006-02-15 09:46:27,931,2006-02-15 10:07:09,Valentine Vanishing,A Thrilling Display of a Husband And a Butler ...,2006,1,7,0.99,48,9.99,PG-13,2013-05-26 14:50:58.951,"[Trailers, Behind the Scenes]",'butler':11 'california':19 'chef':17 'display...
996,16,Travel,2006-02-15 09:46:27,977,2006-02-15 10:07:09,Window Side,A Astounding Character Study of a Womanizer An...,2006,1,3,2.99,85,25.99,R,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'astound':4 'charact':5 'escap':15 'hunter':12...
997,16,Travel,2006-02-15 09:46:27,981,2006-02-15 10:07:09,Wolves Desire,A Fast-Paced Drama of a Squirrel And a Robot w...,2006,1,7,0.99,55,13.99,NC-17,2013-05-26 14:50:58.951,[Behind the Scenes],'desir':2 'drama':7 'fast':5 'fast-pac':4 'man...
998,16,Travel,2006-02-15 09:46:27,988,2006-02-15 10:07:09,Worker Tarzan,A Action-Packed Yarn of a Secret Agent And a T...,2006,1,7,2.99,139,26.99,R,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Behind the Scenes]",'action':5 'action-pack':4 'agent':11 'battl':...


In [190]:
film_actor_data.head()

,actor_id,film_id,last_update
0,1,1,2006-02-15 10:05:03
1,1,23,2006-02-15 10:05:03
2,1,25,2006-02-15 10:05:03
3,1,106,2006-02-15 10:05:03
4,1,140,2006-02-15 10:05:03


In [191]:
actor_data.head()

,actor_id,first_name,last_name,last_update
0,1,Penelope,Guiness,2013-05-26 14:47:57.620
1,2,Nick,Wahlberg,2013-05-26 14:47:57.620
2,3,Ed,Chase,2013-05-26 14:47:57.620
3,4,Jennifer,Davis,2013-05-26 14:47:57.620
4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620


In [192]:
film_actor_detail = film_actor_data.merge(actor_data, how='left', on='actor_id', suffixes=('_x3', '_y3'))
film_actor_detail

,actor_id,film_id,last_update_x3,first_name,last_name,last_update_y3
0,1,1,2006-02-15 10:05:03,Penelope,Guiness,2013-05-26 14:47:57.620
1,1,23,2006-02-15 10:05:03,Penelope,Guiness,2013-05-26 14:47:57.620
2,1,25,2006-02-15 10:05:03,Penelope,Guiness,2013-05-26 14:47:57.620
3,1,106,2006-02-15 10:05:03,Penelope,Guiness,2013-05-26 14:47:57.620
4,1,140,2006-02-15 10:05:03,Penelope,Guiness,2013-05-26 14:47:57.620
...,...,...,...,...,...,...
5457,200,879,2006-02-15 10:05:03,Thora,Temple,2013-05-26 14:47:57.620
5458,200,912,2006-02-15 10:05:03,Thora,Temple,2013-05-26 14:47:57.620
5459,200,945,2006-02-15 10:05:03,Thora,Temple,2013-05-26 14:47:57.620
5460,200,958,2006-02-15 10:05:03,Thora,Temple,2013-05-26 14:47:57.620


In [ ]:
film_actor_detail[film_actor_detail['film_id']==1]

,actor_id,film_id,last_update_x3,first_name,last_name,last_update_y3
0,1,1,2006-02-15 10:05:03,Penelope,Guiness,2013-05-26 14:47:57.620
212,10,1,2006-02-15 10:05:03,Christian,Gable,2013-05-26 14:47:57.620
489,20,1,2006-02-15 10:05:03,Lucille,Tracy,2013-05-26 14:47:57.620
780,30,1,2006-02-15 10:05:03,Sandra,Peck,2013-05-26 14:47:57.620
1021,40,1,2006-02-15 10:05:03,Johnny,Cage,2013-05-26 14:47:57.620
1378,53,1,2006-02-15 10:05:03,Mena,Temple,2013-05-26 14:47:57.620
2932,108,1,2006-02-15 10:05:03,Warren,Nolte,2013-05-26 14:47:57.620
4428,162,1,2006-02-15 10:05:03,Oprah,Kilmer,2013-05-26 14:47:57.620
5112,188,1,2006-02-15 10:05:03,Rock,Dukakis,2013-05-26 14:47:57.620
5387,198,1,2006-02-15 10:05:03,Mary,Keitel,2013-05-26 14:47:57.620


In [196]:
film_actor_detail['full_name'] = film_actor_detail['first_name'] + ' ' + film_actor_detail['last_name']

In [197]:
film_actor_detail['full_name']

0       Penelope Guiness
1       Penelope Guiness
2       Penelope Guiness
3       Penelope Guiness
4       Penelope Guiness
              ...       
5457        Thora Temple
5458        Thora Temple
5459        Thora Temple
5460        Thora Temple
5461        Thora Temple
Name: full_name, Length: 5462, dtype: object

In [210]:
film_actor_detail = film_actor_detail.groupby(['film_id', 'last_update_x3', 'last_update_y3'])['full_name'].apply(lambda x: ', '.join(x))
film_actor_detail

film_id  last_update_x3       last_update_y3         
1        2006-02-15 10:05:03  2013-05-26 14:47:57.620    Penelope Guiness, Christian Gable, Lucille Tra...
2        2006-02-15 10:05:03  2013-05-26 14:47:57.620    Bob Fawcett, Minnie Zellweger, Sean Guiness, C...
3        2006-02-15 10:05:03  2013-05-26 14:47:57.620    Nick Wahlberg, Bob Fawcett, Cameron Streep, Ra...
4        2006-02-15 10:05:03  2013-05-26 14:47:57.620    Jodie Degeneres, Scarlett Damon, Kenneth Pesci...
5        2006-02-15 10:05:03  2013-05-26 14:47:57.620    Gary Phoenix, Dustin Tautou, Matthew Leigh, Ma...
                                                                               ...                        
996      2006-02-15 10:05:03  2013-05-26 14:47:57.620    Ed Chase, Julia Mcqueen, James Pitt, Christoph...
997      2006-02-15 10:05:03  2013-05-26 14:47:57.620    Sandra Kilmer, Val Bolger, Scarlett Bening, Ia...
998      2006-02-15 10:05:03  2013-05-26 14:47:57.620    Uma Wood, Nick Stallone, Gary Pen

In [211]:
film_actor_detail = pd.DataFrame(film_actor_detail)
film_actor_detail

,,,full_name
film_id,last_update_x3,last_update_y3,
1,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Penelope Guiness, Christian Gable, Lucille Tra..."
2,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Bob Fawcett, Minnie Zellweger, Sean Guiness, C..."
3,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Nick Wahlberg, Bob Fawcett, Cameron Streep, Ra..."
4,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Jodie Degeneres, Scarlett Damon, Kenneth Pesci..."
5,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Gary Phoenix, Dustin Tautou, Matthew Leigh, Ma..."
...,...,...,...
996,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Ed Chase, Julia Mcqueen, James Pitt, Christoph..."
997,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Sandra Kilmer, Val Bolger, Scarlett Bening, Ia..."
998,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Uma Wood, Nick Stallone, Gary Penn, Salma Nolt..."


In [212]:
film_actor_detail = film_actor_detail.reset_index()
film_actor_detail

,film_id,last_update_x3,last_update_y3,full_name
0,1,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Penelope Guiness, Christian Gable, Lucille Tra..."
1,2,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Bob Fawcett, Minnie Zellweger, Sean Guiness, C..."
2,3,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Nick Wahlberg, Bob Fawcett, Cameron Streep, Ra..."
3,4,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Jodie Degeneres, Scarlett Damon, Kenneth Pesci..."
4,5,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Gary Phoenix, Dustin Tautou, Matthew Leigh, Ma..."
...,...,...,...,...
992,996,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Ed Chase, Julia Mcqueen, James Pitt, Christoph..."
993,997,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Sandra Kilmer, Val Bolger, Scarlett Bening, Ia..."
994,998,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Uma Wood, Nick Stallone, Gary Penn, Salma Nolt..."
995,999,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Carmen Hunt, Mary Tandy, Penelope Cronyn, Whoo..."


In [214]:
film_list = film_list.merge(film_actor_detail, how='left', on='film_id', suffixes=('_x4', '_y4'))
film_list

,category_id,name,last_update_x1,film_id,last_update_y1,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext,last_update_x3,last_update_y3,full_name
0,1,Action,2006-02-15 09:46:27,19,2006-02-15 10:07:09,Amadeus Holy,A Emotional Display of a Pioneer And a Technic...,2006,1,6,0.99,113,20.99,PG,2013-05-26 14:50:58.951,"[Commentaries, Deleted Scenes, Behind the Scenes]",'amadeus':1 'baloon':20 'battl':15 'display':5...,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Johnny Lollobrigida, Julia Mcqueen, Val Bolger..."
1,1,Action,2006-02-15 09:46:27,21,2006-02-15 10:07:09,American Circus,A Insightful Drama of a Girl And a Astronaut w...,2006,1,3,4.99,129,17.99,R,2013-05-26 14:50:58.951,"[Commentaries, Behind the Scenes]",'administr':17 'american':1 'astronaut':11 'ci...,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Kevin Bloom, Rip Crawford, Sidney Crowe, Warre..."
2,1,Action,2006-02-15 09:46:27,29,2006-02-15 10:07:09,Antitrust Tomatoes,A Fateful Yarn of a Womanizer And a Feminist w...,2006,1,5,2.99,168,11.99,NC-17,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Deleted Scenes]",'administr':17 'ancient':19 'antitrust':1 'dat...,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Bette Nicholson, Uma Wood, Sidney Crowe, Salma..."
3,1,Action,2006-02-15 09:46:27,38,2006-02-15 10:07:09,Ark Ridgemont,A Beautiful Yarn of a Pioneer And a Monkey who...,2006,1,6,0.99,68,25.99,NC-17,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Deleted Scenes, Behin...",'ark':1 'beauti':4 'desert':20 'explor':16 'mo...,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Parker Goldberg, Nick Degeneres, Audrey Bailey"
4,1,Action,2006-02-15 09:46:27,56,2006-02-15 10:07:09,Barefoot Manchurian,A Intrepid Story of a Cat And a Student who mu...,2006,1,6,2.99,129,15.99,G,2013-05-26 14:50:58.951,"[Trailers, Commentaries]",'abandon':19 'amus':20 'barefoot':1 'cat':8 'g...,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Jennifer Davis, Elvis Marx, Milla Peck, Kirk J..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,16,Travel,2006-02-15 09:46:27,931,2006-02-15 10:07:09,Valentine Vanishing,A Thrilling Display of a Husband And a Butler ...,2006,1,7,0.99,48,9.99,PG-13,2013-05-26 14:50:58.951,"[Trailers, Behind the Scenes]",'butler':11 'california':19 'chef':17 'display...,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Burt Dukakis, Ed Guiness"
996,16,Travel,2006-02-15 09:46:27,977,2006-02-15 10:07:09,Window Side,A Astounding Character Study of a Womanizer An...,2006,1,3,2.99,85,25.99,R,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'astound':4 'charact':5 'escap':15 'hunter':12...,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Lucille Tracy, Anne Cronyn, Gina Degeneres, Wi..."
997,16,Travel,2006-02-15 09:46:27,981,2006-02-15 10:07:09,Wolves Desire,A Fast-Paced Drama of a Squirrel And a Robot w...,2006,1,7,0.99,55,13.99,NC-17,2013-05-26 14:50:58.951,[Behind the Scenes],'desir':2 'drama':7 'fast':5 'fast-pac':4 'man...,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Sandra Kilmer, Jayne Neeson, Cameron Wray, Jes..."
998,16,Travel,2006-02-15 09:46:27,988,2006-02-15 10:07:09,Worker Tarzan,A Action-Packed Yarn of a Secret Agent And a T...,2006,1,7,2.99,139,26.99,R,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Behind the Scenes]",'action':5 'action-pack':4 'agent':11 'battl':...,2006-02-15 10:05:03,2013-05-26 14:47:57.620,"Zero Cage, Penelope Pinkett, Jayne Neeson, Sus..."


In [215]:
temp_final = film_list.copy()

In [217]:
film_list = film_list[['film_id', 'title', 'description', 'name', 'rental_rate', 'length', 'rating', 'full_name']].copy()
film_list

,film_id,title,description,name,rental_rate,length,rating,full_name
0,19,Amadeus Holy,A Emotional Display of a Pioneer And a Technic...,Action,0.99,113,PG,"Johnny Lollobrigida, Julia Mcqueen, Val Bolger..."
1,21,American Circus,A Insightful Drama of a Girl And a Astronaut w...,Action,4.99,129,R,"Kevin Bloom, Rip Crawford, Sidney Crowe, Warre..."
2,29,Antitrust Tomatoes,A Fateful Yarn of a Womanizer And a Feminist w...,Action,2.99,168,NC-17,"Bette Nicholson, Uma Wood, Sidney Crowe, Salma..."
3,38,Ark Ridgemont,A Beautiful Yarn of a Pioneer And a Monkey who...,Action,0.99,68,NC-17,"Parker Goldberg, Nick Degeneres, Audrey Bailey"
4,56,Barefoot Manchurian,A Intrepid Story of a Cat And a Student who mu...,Action,2.99,129,G,"Jennifer Davis, Elvis Marx, Milla Peck, Kirk J..."
...,...,...,...,...,...,...,...,...
995,931,Valentine Vanishing,A Thrilling Display of a Husband And a Butler ...,Travel,0.99,48,PG-13,"Burt Dukakis, Ed Guiness"
996,977,Window Side,A Astounding Character Study of a Womanizer An...,Travel,2.99,85,R,"Lucille Tracy, Anne Cronyn, Gina Degeneres, Wi..."
997,981,Wolves Desire,A Fast-Paced Drama of a Squirrel And a Robot w...,Travel,0.99,55,NC-17,"Sandra Kilmer, Jayne Neeson, Cameron Wray, Jes..."
998,988,Worker Tarzan,A Action-Packed Yarn of a Secret Agent And a T...,Travel,2.99,139,R,"Zero Cage, Penelope Pinkett, Jayne Neeson, Sus..."


In [218]:
rename_column_map = {
    'film_id' : 'fid',
    'name' : 'category',
    'rental_rate' : 'price',
    'full_name' : 'actors'
}

In [219]:
film_list.rename(columns=rename_column_map, inplace=True)
film_list

,fid,title,description,category,price,length,rating,actors
0,19,Amadeus Holy,A Emotional Display of a Pioneer And a Technic...,Action,0.99,113,PG,"Johnny Lollobrigida, Julia Mcqueen, Val Bolger..."
1,21,American Circus,A Insightful Drama of a Girl And a Astronaut w...,Action,4.99,129,R,"Kevin Bloom, Rip Crawford, Sidney Crowe, Warre..."
2,29,Antitrust Tomatoes,A Fateful Yarn of a Womanizer And a Feminist w...,Action,2.99,168,NC-17,"Bette Nicholson, Uma Wood, Sidney Crowe, Salma..."
3,38,Ark Ridgemont,A Beautiful Yarn of a Pioneer And a Monkey who...,Action,0.99,68,NC-17,"Parker Goldberg, Nick Degeneres, Audrey Bailey"
4,56,Barefoot Manchurian,A Intrepid Story of a Cat And a Student who mu...,Action,2.99,129,G,"Jennifer Davis, Elvis Marx, Milla Peck, Kirk J..."
...,...,...,...,...,...,...,...,...
995,931,Valentine Vanishing,A Thrilling Display of a Husband And a Butler ...,Travel,0.99,48,PG-13,"Burt Dukakis, Ed Guiness"
996,977,Window Side,A Astounding Character Study of a Womanizer An...,Travel,2.99,85,R,"Lucille Tracy, Anne Cronyn, Gina Degeneres, Wi..."
997,981,Wolves Desire,A Fast-Paced Drama of a Squirrel And a Robot w...,Travel,0.99,55,NC-17,"Sandra Kilmer, Jayne Neeson, Cameron Wray, Jes..."
998,988,Worker Tarzan,A Action-Packed Yarn of a Secret Agent And a T...,Travel,2.99,139,R,"Zero Cage, Penelope Pinkett, Jayne Neeson, Sus..."


## Load Data to Data Warehouse

In [ ]:
def postgres_engine(db_name):
    user = os.getenv("USER_DATA_DESTINATION")
    password = os.getenv("PASSWORD_DATA_DESTINATION")
    host = os.getenv("HOST_DATA_DESTINATION")
    port = os.getenv("PORT_DATA_DESTINATION")
    
    engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db_name}")

    try:
        # Tes koneksi
        with engine.connect() as conn:
            print("Connection successful.")
    except Exception as e:
        print("Connection failed.")
        print(e)
    
    return engine

In [ ]:
engine = postgres_engine(db_name=os.getenv("DB_NAME_DATA_DESTINATION"))

Connection successful.


In [224]:
film_list.to_sql('film_list', engine, if_exists='replace', index=False)

1000

In [228]:
df = pd.read_sql("SELECT * FROM film_list LIMIT 5", engine)
df

,fid,title,description,category,price,length,rating,actors
0,19,Amadeus Holy,A Emotional Display of a Pioneer And a Technic...,Action,0.99,113,PG,"Johnny Lollobrigida, Julia Mcqueen, Val Bolger..."
1,21,American Circus,A Insightful Drama of a Girl And a Astronaut w...,Action,4.99,129,R,"Kevin Bloom, Rip Crawford, Sidney Crowe, Warre..."
2,29,Antitrust Tomatoes,A Fateful Yarn of a Womanizer And a Feminist w...,Action,2.99,168,NC-17,"Bette Nicholson, Uma Wood, Sidney Crowe, Salma..."
3,38,Ark Ridgemont,A Beautiful Yarn of a Pioneer And a Monkey who...,Action,0.99,68,NC-17,"Parker Goldberg, Nick Degeneres, Audrey Bailey"
4,56,Barefoot Manchurian,A Intrepid Story of a Cat And a Student who mu...,Action,2.99,129,G,"Jennifer Davis, Elvis Marx, Milla Peck, Kirk J..."
